# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            print(dt.hour, dt.minute)
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 4800


  0%|                                                                                                                                                                                                 | 0/4800 [00:00<?, ?it/s]


Load the datasets...
Binance
BinanceFutures


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4800/4800 [2:12:03<00:00,  1.65s/it]


# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
stat.groupby(['RSI_timeperiod', 
              'RSI_low_bound', 
              'STOCH_fastk_period', 
              'STOCH_slowk_period', 
              'STOCH_slowd_period', 
              'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                       'pct_price_diff_avg': 'mean',
                                       'forecast_rank': 'mean', 
                                       'price_rank': 'mean', 
                                       'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                            ascending=False).head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             35            11                 3                  9                  25                            72.761000   
                                                                                      20                            73.122833   
12             30            11                 3                  9                  25                            72.232833   
14             30            11                 2                  3                  25                            69.037333   
16             30            5                  4                  3                  10                            82.471667   
                                                                   5                  10                            96.550787   
               35            11                 5                  9                  25                            71.573750   
               25            7                  3                  5                  10                            99.305556   
12             20            7                  3                  5                  10                            97.531065   
14             30            11                 5                  3                  20                            68.214750   
                                                3                  9                  25                            71.490167   
12             25            5                  4                  5                  10                            89.227361   
14             30            5                  4                  5                  10                            80.673935   
               25            5                  4                  5                  10                            98.572546   
16             25            5                  4                  3                  10                            91.705602   
12             30            11                 3                  9                  20                            68.972333   
14             25            5                  4                  3                  10                            85.591759   
16             35            11                 5                  5                  20                            68.022167   
12             20            5                  4                  5                  10                           100.000000   
16             25            5                  4                  5                  10                           100.000000   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             35            11                 3                  9                  25                         0.652583   
                                                                                      20                         0.685333   
12             30            11                 3                  9                  25                         0.579250   
14             30            11                 2                  3                  25                         0.584167   
16             30            5                  4                  3                  10                         1.064213   
                                                                   5                  10                         1.908426   
               35            11                 5                  9                  25                         0.570250   
               25            7                  3                  5                  10                         2.435463   
12            

# Save new config data to config file

In [5]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [30]},
                  'STOCH': {'fastk_period': [11], 'slowk_period': [3],
                            'slowd_period': [9], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)